<a href="https://colab.research.google.com/github/VarunB1234/Depression-Detection/blob/main/Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

def processData(data):
    X = data.iloc[:, :].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if isinstance(X[i][5], str) or isinstance(X[i][7], str):
            X[i] = np.zeros((1, X.shape[1]))
    return X

def getData(filename):
    data = pd.read_csv(filename, delimiter=',', engine='python')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo) + "_CLNF_AUs.txt"
    file2 = (patientNo) + "_CLNF_features.txt"
    file3 = (patientNo) + "_CLNF_features3D.txt"
    file4 = (patientNo) + "_CLNF_gaze.txt"
    file5 = (patientNo) + "_CLNF_hog.txt"
    file6 = (patientNo) + "_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), axis=1)
    return X

def scale_down(X):
    X_new = []
    size = 2
    for i in range(int(X.shape[0]/size)):
        cur_row = X[i*size]
        for j in range(1, size):
            if i + j < X.shape[0]:
                cur_row += X[i + j]
        cur_row = cur_row / size
        X_new.append(cur_row)
    return np.array(X_new)

def decrease_size(X):
    size = 10000
    if X.shape[0] < size:
        pad = np.zeros((size - X.shape[0], X.shape[1]))
        X = np.concatenate((X, pad), axis=0)
    else:
        X = X[:size, :]
    return X

def makeDataset(location, folder):
    file = np.array(pd.read_csv(location, delimiter=',', encoding='utf-8'))[:, 0:2]
    X_temp = []
    Y_temp = []
    for i in range(len(file)):
        patientID = str(int(file[i][0]))
        path = '/content/drive/My Drive/MCA Dataset/' + folder + '/' + patientID
        XT = makeDataPoint(path)
        XT = scale_down(XT)
        XT = decrease_size(XT)
        X_temp.append(XT)
        Y_temp.append(int(file[i][1]))
    return X_temp, np.asarray(Y_temp)


In [2]:
X_train, Y_train = makeDataset('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', 'train_data')
X_dev, Y_dev = makeDataset('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', 'dev_data')


In [3]:
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense

class CNN_Video:
    def __init__(self):
        classifier = Sequential()
        classifier.add(Input(shape=(10000, 388)))
        classifier.add(Conv1D(300, 3, activation='relu'))
        classifier.add(MaxPooling1D(pool_size=2))
        classifier.add(Conv1D(150, 3, activation='relu'))
        classifier.add(MaxPooling1D(pool_size=2))
        classifier.add(Conv1D(75, 3, activation='relu'))
        classifier.add(MaxPooling1D(pool_size=2))
        classifier.add(Conv1D(32, 3, activation='relu'))
        classifier.add(MaxPooling1D(pool_size=2))
        classifier.add(Flatten())
        classifier.add(Dense(128, activation='relu'))
        classifier.add(Dense(1, activation='sigmoid'))
        classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        self.classifier = classifier

    def fitModel(self, X_train, Y_train, epoch=5):
        return self.classifier.fit(X_train, Y_train, epochs=epoch)

    def predictModel(self, X_test):
        return self.classifier.predict(X_test)



In [4]:
# Combine them
X_all = X_train + X_dev
Y_all = np.concatenate((Y_train, Y_dev))

# Convert to arrays for stratified split
X_all = np.array(X_all)
Y_all = np.array(Y_all)

from sklearn.model_selection import train_test_split


# === Manual Stratified Train/Validation Split (90% train / 10% val) ===
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(
    X_all, Y_all, test_size=0.1, random_state=42, stratify=Y_all

)

X_train_split = np.array(X_train_split, dtype=np.float32)
X_val_split = np.array(X_val_split, dtype=np.float32)
Y_train_split = np.array(Y_train_split, dtype=np.float32)
Y_val_split = np.array(Y_val_split, dtype=np.float32)

# === Build and Train CNN Model ===
model = CNN_Video()
model.fitModel(X_train_split, Y_train_split, epoch=5)



Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.6572 - loss: 96443.8516
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 6s/step - accuracy: 0.5776 - loss: 48815.3008
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.7105 - loss: 16523.9551
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.5463 - loss: 29445.2344
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.6720 - loss: 13386.4482


In [5]:
# === Predict and Evaluate on Validation Set ===
Y_val_pred_probs = model.predictModel(X_val_split)
Y_val_pred = (Y_val_pred_probs > 0.5).astype(int)
from sklearn.metrics import classification_report
print("Validation Set Evaluation:")
print(classification_report(Y_val_split, Y_val_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Validation Set Evaluation:
              precision    recall  f1-score   support

         0.0       0.73      1.00      0.85        11
         1.0       0.00      0.00      0.00         4

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
